In [28]:
import sys
import re
from random import shuffle

from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score

from nltk.stem import PorterStemmer
from nltk.classify.scikitlearn import SklearnClassifier
import nltk.corpus
from nltk import ngrams, FreqDist

In [2]:
def get_znacilke(komentarji, vse_besede):    
    '''
    Metoda vrne značilke, ki se pojavljajo v posameznem komentarju
    '''    
    urejene_besede = sorted(vse_besede)
    znacilke = []    
    for komentar in komentarji:
        word_map = {}
        for beseda in urejene_besede:
            word_map[beseda] = 0
        for del_komentarja in komentar:
            if(del_komentarja in word_map):
                word_map[del_komentarja] = 1
        values = list(word_map.values())
        znacilke.append(values)
    return znacilke

In [3]:
def get_znacilke_oznake(komentarji, vse_besede):
    '''
    Metoda vrne slovar z tipi komentarji in značilkami, ki se pojavljajo v posameznem komentarju
    '''    
    urejene_besede = sorted(vse_besede)
    znacilke = []
    oznake = []    
    for komentar in komentarji:
        oznaka = 0
        word_map = {}
        for beseda in urejene_besede:
            word_map[beseda] = 0        
        if len(komentar) > 1:
            besedilo_komentarja = komentar[0]
            zaljivost_komentarja = komentar[1]
            for del_komentarja in besedilo_komentarja:
                word_map[del_komentarja] = 1
            values = list(word_map.values())
            znacilke.append(values)
            if zaljivost_komentarja == 0:
                oznaka = 0
            elif zaljivost_komentarja == 1:
                oznaka = 1
            oznake.append(oznaka)        
    return {'pojavitev_besed' : znacilke, 'oznake':oznake}

In [4]:
def preuredi_besedilo(tekst):
    '''
    Metoda vrne prečiščen tekst
    '''
    tekst = tekst.lower()
    tekst = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', tekst)
    tekst = re.sub('newline_token', '', tekst)
    tekst = re.sub('newline', '', tekst)
    tekst = re.sub('tokennewline', '', tekst)
    tekst = re.sub('tab_token', '', tekst)
    tekst = re.sub('``', '"', tekst)
    tekst = re.sub('`', '\'', tekst)
    tekst = re.sub('=', '', tekst)
    tekst = re.sub('\n', '', tekst)
    tekst = re.sub(':', '', tekst)
    tekst = re.sub('[\s]+', ' ', tekst)
    tekst = tekst.strip()
    tekst = tekst.strip('\'"')
    return tekst

In [5]:
def get_bag_of_words(komentar, stopwords):    
    '''
    Metoda odstrani presledke, končnice in vrne posamezne besede. 
    Odstranimo nepotrebne whitespace in spremenimo v male črke
    '''
    komentar = preuredi_besedilo(komentar)
    komentar = " ".join(re.split("[^a-zA-Z]*", komentar.lower())).strip()    
    stemmer = PorterStemmer()
    besede = [] 
    for beseda in komentar.split():
        if not beseda in stopwords:
            try:
                besede.append(stemmer.stem(beseda))
            except(IndexError):
                besede.append(beseda)
    return besede

In [6]:
def word2ngrams(text, n=3, exact=True):
    '''
    Vrne crkovne ngrame nad besedami
    '''
    return ["".join(j) for j in zip(*[text[i:] for i in range(n)])]

In [7]:
def get_ngram(komentar, min_n=1, max_n=3, stopwords=None, join=True):
    '''
    Metoda vrne max_n - min_n razlicnih tipove ngramov 
    '''
    if min_n > max_n:
        return get_ngram(komentar, max_n, min_n, stopwords, join)
    cngrams = []
    besede = ' '.join(get_bag_of_words(komentar, stopwords)).split()
    for i in range(min_n, max_n + 1):
        cngrams.append(ngrams(besede, i))
    if join: 
        return [e for ngram in cngrams for e in list(ngram)]
    else:
        return cngrams

In [8]:
def get_all_words(tekst, algoritem, stopwords):
    rdata = []
    for comment in tekst:
        comment = comment.split('\t')
        offens_type = int(comment[0])
        comment = comment[1]
        for e in algoritem(komentar=comment, stopwords=stopwords):
            rdata.append(e)
    return rdata

In [18]:
def convert_data(data, algoritem, stopwords, word_features, joinset=True):
    x, y = [], []
    for comment in data:
        comment = comment.split('\t')
        offens_type = int(comment[0])
        comment = comment[1]
        tmp = {}
        for e in algoritem(komentar=comment, stopwords=stopwords):
            tmp[e] = (e in word_features)
        x.append(tmp), y.append(offens_type)
    if joinset:
        out = []
        for i in range(len(y)):
            out.append((x[i], y[i]))
        return out
    else:
        return x, y

In [10]:
def get_predict_data(data):
    x = []
    y = []
    for e in data:
        x.append(e[0])
        y.append(e[1])
    return x, y

In [ ]:
def run(tekst, test_tekst, features_proc, model, algoritem):
    stopwords = nltk.corpus.stopwords.words("english")
    all_words = []
    for w in get_all_words(tekst, algoritem, stopwords):
        all_words.append(w)
    all_words = FreqDist(all_words)
    word_features = list(all_words.keys())[:int(len(all_words) * features_proc)]
    x = convert_data(tekst, algoritem, stopwords, word_features)
    x_t, y_t = convert_data(test_tekst, algoritem, stopwords, word_features, False)
    classifier = SklearnClassifier(model).train(x)
    y_p = classifier.classify_many(x_t)
    print("SVC_classifier accuracy percent:",
          (accuracy_score(y_t, y_p)))
    print(classification_report(y_t, y_p))

In [38]:
TRAIN_SET_PER = 0.7
CORP_SIZE = 1200

def main():
    datoteka = open('comments.txt', encoding="ISO-8859-1")
    tekst = datoteka.readlines()[1:CORP_SIZE + 1]
    datoteka.close()    
    shuffle(tekst)    
    st_komentarjev = len(tekst)
    deli = int(st_komentarjev * TRAIN_SET_PER)       
    ucna_tekst = tekst[:deli]    
    test_tekst = tekst[deli:st_komentarjev]    
    run(ucna_tekst, test_tekst, 0.3, svm.SVC(), get_bag_of_words)
    run(ucna_tekst, test_tekst, 0.3, svm.SVC(), get_ngram)

In [ ]:
main()

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
/usr/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: generator 'ngrams' raised StopIteration
